In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, optimizers, Sequential, metrics

# 下载minist数据
(xs, ys),_ = datasets.mnist.load_data()
print('datasets:', xs.shape, ys.shape, xs.min(), xs.max())



11490434/11490434 [==============================] - 4s 0us/step
datasets: (60000, 28, 28) (60000,) 0 255


In [2]:
# 处理数据

xs = tf.convert_to_tensor(xs, dtype=tf.float32) / 255.
db = tf.data.Dataset.from_tensor_slices((xs,ys))
db = db.batch(32).repeat(10)

In [3]:
# 构建网络
network = Sequential([layers.Dense(256, activation='relu'),
                     layers.Dense(256, activation='relu'),
                     layers.Dense(256, activation='relu'),
                     layers.Dense(10)])
network.build(input_shape=(None, 28*28))
network.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               200960    
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 10)                2570      
                                                                 
Total params: 335,114
Trainable params: 335,114
Non-trainable params: 0
_________________________________________________________________


In [4]:
# 训练
optimizer = optimizers.SGD(lr=0.01)
acc_meter = metrics.Accuracy()

for step, (x,y) in enumerate(db):

    with tf.GradientTape() as tape:
        # [b, 28, 28] => [b, 784]
        x = tf.reshape(x, (-1, 28*28))
        # [b, 784] => [b, 10]
        out = network(x)
        # [b] => [b, 10]
        y_onehot = tf.one_hot(y, depth=10)
        # [b, 10]
        loss = tf.square(out-y_onehot)
        # [b]
        loss = tf.reduce_sum(loss) / 32


    acc_meter.update_state(tf.argmax(out, axis=1), y)

    grads = tape.gradient(loss, network.trainable_variables)
    optimizer.apply_gradients(zip(grads, network.trainable_variables))


    if step % 200==0:

        print(step, 'loss:', float(loss), 'acc:', acc_meter.result().numpy())
        acc_meter.reset_states()

f:\Code\python\AILearn\env\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


0 loss: 1.5347081422805786 acc: 0.15625
200 loss: 0.42675477266311646 acc: 0.6875
400 loss: 0.3711259067058563 acc: 0.8421875
600 loss: 0.32421213388442993 acc: 0.865
800 loss: 0.261374294757843 acc: 0.8957813
1000 loss: 0.2811684012413025 acc: 0.8990625
1200 loss: 0.28251758217811584 acc: 0.91203123
1400 loss: 0.22117739915847778 acc: 0.9139063
1600 loss: 0.21573254466056824 acc: 0.91125
1800 loss: 0.1878446489572525 acc: 0.9298437
2000 loss: 0.23168635368347168 acc: 0.9409375
2200 loss: 0.13439443707466125 acc: 0.93078125
2400 loss: 0.21195188164710999 acc: 0.92859375
2600 loss: 0.19127200543880463 acc: 0.9375
2800 loss: 0.13681891560554504 acc: 0.938125
3000 loss: 0.2136998474597931 acc: 0.936875
3200 loss: 0.17318466305732727 acc: 0.936875
3400 loss: 0.1528593897819519 acc: 0.93625
3600 loss: 0.125424325466156 acc: 0.93875
3800 loss: 0.15749666094779968 acc: 0.9560937
4000 loss: 0.20749050378799438 acc: 0.9529688
4200 loss: 0.12807586789131165 acc: 0.9428125
4400 loss: 0.1629419475